In [30]:
import os
import requests
import pandas as pd 
import datetime as dt 
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secrets_config import db_user, db_password, db_server_name, db_database_name
from sqlalchemy.schema import CreateTable 

In [31]:
# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)

engine = create_engine(connection_url)

In [42]:
df = pd.read_csv('data/new_example_output_extract.csv')
df = df.drop(columns='Unnamed: 0')
df.head()

,pull_date,airport_code,departures_onTime,departures_delayed,departures_canceled,arrivals_onTime,arrivals_delayed,arrivals_canceled
0,2023-02-06,TFS,91,44,0,114,14,0
1,2023-02-06,IAD,175,152,1,278,46,1
2,2023-02-06,WAW,58,131,0,145,31,1


In [33]:
current_timestamp = dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
df.to_parquet(f"data/flights_{current_timestamp}.parquet")

In [7]:
#df.to_parquet("data/flights.parquet")

In [ ]:
# # upsert (update and insert) data to a csv file 
# if "flights.parquet" in os.listdir("data/"): 
#     df_current = pd.read_parquet("data/flights.parquet")
#     df_concat = pd.concat(objs=[df_current,df_selected[~df_selected.index.isin(df_current.index)]]) # ~: converts true to false, and false to true. 
#     df_concat.to_parquet("data/flights.parquet")
# else:
#     df_selected.to_parquet("data/flights.parquet")

In [34]:
meta = MetaData()
flights_table = Table(
    "flights", meta, 
    Column("pull_date", String, primary_key=True),
    Column("airport_code", String, primary_key=True),
    Column("departures_onTime", Integer),
    Column("departures_delayed", Integer),
    Column("departures_canceled", Integer),
    Column("arrivals_onTime", Integer),
    Column("arrivals_delayed", Integer),
    Column("arrivals_canceled", Integer)
)
meta.create_all(engine) # creates table if it does not exist 

In [35]:
insert_statement = postgresql.insert(flights_table).values(df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['pull_date', 'airport_code'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['pull_date', 'airport_code']})
engine.execute(upsert_statement)